# Information Retrieval Baseline Code

## 대회 설명
Information Retrieval 경진대회에 오신 여러분 환영합니다! 🎉 <br>
본 대회에서는 과학 상식에 대한 멀터턴 질문에 대하여 RAG 기술을 활용하여 질문에 적합한 레퍼런스를 추출하고 이를 토대로 답변을 생성하는 어플리케이션을 구현해 봅니다. <br>
LLM은 GPT-3.5 버전을 이용하고, 레퍼런스 추출을 위한 검색엔진 구축은 Elasticsearch를 활용합니다.

## 데이터셋 준비
* 질문에 대한 레퍼런스로 활용될 수 있는 4272개의 과학 상식을 다루는 문서
 * Open Ko LLM Leaderboard에 들어가는 Ko-H4 데이터 중 MMLU, ARC 데이터를 기반으로 생성
 * Question과 answer를 paraphrase (question과 answer를 토대로 GPT-4 활용하여 document 생성)
 * 예시)
  ```
  수소, 산소, 질소 가스의 혼합물에서 평균 속도가 가장 빠른 분자는 수소입니다
. 수소 분자는 가장 가볍고 작은 원자로 구성되어 있기 때문에 다른 분자들보다 더 빠르게 움직>입니다. 이러한 이유로 수소 분자는 주어진 온도에서 가장 빠른 평균 속도를 가지고 있습니다. >수소 분자는 화학 반응에서도 활발하게 참여하며, 수소 연료로도 널리 사용됩니다. 따라서 수소 분자는 주어진 온도에서 평균 속도가 가장 빠른 분자입니다.
  ```

* 가상의 질문 220개
 * 과학 상식 질문 또는 일상 대화 메시지를 수동으로 생성
 * 220개 중 20개는 멀티턴 대화 형태의 질문, 20개는 일상 칫챗 대화
  * 예시)
  ```
{"eval_id": 0, "msg": [{"role": "user", "content": "나무의 분류에 대해 조사해 보기 위한 방
법은?"}], "refs": ["c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d"]}
{"eval_id": 1, "msg": [{"role": "user", "content": "각 나라에서의 공교육 지출 현황에 대해 알려줘."}], "refs": ["79c93deb-fe60-4c81-8d51-cb7400a0a156"]}
{"eval_id": 2, "msg": [{"role": "user", "content": "기억 상실증 걸리면 너무 무섭겠다."}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "어떤 원인 때
문에 발생하는지 궁금해."}], "refs": ["25de4ffd-cee4-4f27-907e-fd6b802c6ede"]}
  ```

21 : {"eval_id": 276, "msg": [{"role": "user", "content": "요새 너무 힘들다."}]}
36 : {"eval_id": 261, "msg": [{"role": "user", "content": "니가 대답을 잘해줘서 너무 신나!"}]}
51 : {"eval_id": 283, "msg": [{"role": "user", "content": "이제 그만 얘기해!"}]}
52 : {"eval_id": 32, "msg": [{"role": "user", "content": "오늘 너무 즐거웠다!"}]}
53 : {"eval_id": 94, "msg": [{"role": "user", "content": "우울한데 신나는 얘기 좀 해줘!"}]}
64 : {"eval_id": 90, "msg": [{"role": "user", "content": "안녕 반갑다"}]}
89 : {"eval_id": 220, "msg": [{"role": "user", "content": "너는 누구야?"}]}
94 : {"eval_id": 245, "msg": [{"role": "user", "content": "너 모르는 것도 있니?"}]}
99 : {"eval_id": 229, "msg": [{"role": "user", "content": "너 잘하는게 뭐야?"}]}
102 : {"eval_id": 247, "msg": [{"role": "user", "content": "너 정말 똑똑하다!"}]}
115 : {"eval_id": 67, "msg": [{"role": "user", "content": "니가 대답을 잘해줘서 기분이 좋아!"}]}
117 : {"eval_id": 57, "msg": [{"role": "user", "content": "우울한데 신나는 얘기 좀 해줄래?"}]}
118 : {"eval_id": 2, "msg": [{"role": "user", "content": "이제 그만 얘기하자."}]}

124 : {"eval_id": 227, "msg": [{"role": "user", "content": "너는 누구니?"}]}
133 : {"eval_id": 301, "msg": [{"role": "user", "content": "오늘 너무 즐거웠어!"}]}
148 : {"eval_id": 222, "msg": [{"role": "user", "content": "안녕 반가워"}]}
151 : {"eval_id": 83, "msg": [{"role": "user", "content": "너 정말 똑똑하구나?"}]}
161 : {"eval_id": 64, "msg": [{"role": "user", "content": "너 모르는 것도 있어?"}]}
175 : {"eval_id": 103, "msg": [{"role": "user", "content": "너 뭘 잘해?"}]}
181 : {"eval_id": 218, "msg": [{"role": "user", "content": "요새 너무 힘드네.."}]}

In [1]:
#  # 색인 대상 문서 및 평가 데이터 다운로드
#  !wget https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000291/data/data.tar.gz
#  !tar -xzvf data.tar.gz

--2024-02-14 05:15:10--  https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000291/data/data.tar.gz
Resolving aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)... 52.219.206.95, 52.219.202.27, 52.219.56.70, ...
Connecting to aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)|52.219.206.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1068264 (1.0M) [binary/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   1.02M  1.49MB/s    in 0.7s    

2024-02-14 05:15:12 (1.49 MB/s) - ‘data.tar.gz’ saved [1068264/1068264]

data/
data/eval.jsonl
data/documents.jsonl


## 환경 설정
검색엔진을 위한 Elasitcsearch, 임베딩 생성을 위한 sentence transformers, LLM 사용을 위한 openai client 설치가 필요합니다.

In [10]:
# # Elasticsearch 8.8.0 다운로드 및 압축 해제
# !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
# !tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz

# # daemon user로 구동을 하기 위해 소유자 변경
# !chown -R daemon:daemon elasticsearch-8.8.0/

# # # 코랩 노트북 환경에서 서버 구동을 위한 리소스 제한/격리를 위해 아래 명령 수행
# # !umount /sys/fs/cgroup
# # !apt install cgroup-tools

# # Nori 형태소 분석기 설치
# !./elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori

--2024-04-24 15:29:24--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 596757716 (569M) [application/x-gzip]
Saving to: ‘elasticsearch-8.8.0-linux-x86_64.tar.gz’

elasticsearch-8.8.0 100%[===================>] 569.11M   110MB/s    in 5.2s    

2024-04-24 15:29:30 (110 MB/s) - ‘elasticsearch-8.8.0-linux-x86_64.tar.gz’ saved [596757716/596757716]

-> Installing analysis-nori
-> Downloading analysis-nori from elastic
[=================================================] 100%   
-> Installed analysis-nori
-> Please restart Elasticsearch to activate any plugins installed


In [3]:
# # Elasticsearch python 패키지 설치
# !pip install elasticsearch==8.8.0

# # OpenAI Python 패키지 설치
# !pip install openai==1.7.2

# # 임베딩 생성을 위한 벡터 인코더 설치
# !pip install sentence-transformers==2.2.2

## 검색엔진 준비 - Elasticsearch




In [93]:
# 엘라스틱서치의 데몬 인스턴스 만들기
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(60)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [96]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [97]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon   1030824 1026150  1 07:55 ?        00:00:02 /data/ephemeral/home/IR/code/elasticsearch-8.8.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=elasticsearch-8.8.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/data/ephemeral/home/IR/code/elasticsearch-8.8.0 -Des.path.conf=/data/ephemeral/home/IR/code/elasticsearch-8.8.0/config -Des.distribution.type=tar -cp /data/ephemeral/home/IR/code/elasticsearch-8.8.0/lib/*:/data/ephemeral/home/IR/code/elasticsearch-8.8.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
daemon   1030892 1030824 41 07:55 ?        00:00:57 /data/ephemeral/home/IR/code/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycle

In [98]:
# 데몬 구동후 password 설정 단계 필요
# 명령 실행 후 "Please confirm that you would like to continue"에서 y 입력 필요
# !echo "y" |./elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

In [99]:
es_username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
es_password = ''

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="./elasticsearch-8.8.0/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'instance-8347', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'HLF1ngPSQ1WjS7h2llI_Lw', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [100]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
# model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
# model = SentenceTransformer("bespin-global/klue-sroberta-base-continue-learning-by-mnr")
# model = SentenceTransformer("jhgan/ko-sroberta-multitask")
# model = SentenceTransformer("jhgan/ko-sbert-sts")
# model = SentenceTransformer("bongsoo/moco-sentencedistilbertV2.1")
model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")


# SetntenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str,
                "boost": 0.035             
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 50,
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)

def mix_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    body = {
        "query": {
            "match": {
                "content": {
                    "query": query_str,
                    "boost": 0.035
                }
            }
        },
        "knn": {
            "field": "embeddings",
            "query_vector": query_embedding.tolist(),
            "k": 5,
            "num_candidates": 50,
            "boost": 1
        },
        "size" : size
    }


    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", body=body)

In [116]:
# 색인을 위한 setting 설정
settings = {
    "index": {
        "similarity": {
            "lm_jelinek_mercer": { 
                "type": "LMJelinekMercer", 
                "lambda": 0.7 
            } 
        }
    },
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "SSC", "SSO", "SY", "VCN", "VCP", "VX"]
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        "content": {"type": "text", 
                    "analyzer": "nori", 
                    "similarity": "lm_jelinek_mercer"},
        "embeddings": {
            "type": "dense_vector",
            "dims": 512, #768 # baseline
            "index": True,
            "similarity": "l2_norm"
        }
    }
}


In [114]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

In [115]:
# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)


batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200


BulkIndexError: 500 document(s) failed to index.

In [ ]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

In [ ]:
# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.66356814 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 0.42161518 source: 메릴랜드 Space Grant Observatory는 볼티모어에 위치해 있습니다. 학생들은 이 망원경을 방문하여 별, 행성, 달에 대해 배웠습니다. 그들은 아래와 같은 정보를 기록했습니다. 첫째, 별 패턴은 그대로 유지되지만, 하늘에서의 위치는 변하는 것처럼 보입니다. 둘째, 태양, 행성, 달은 하늘에서 움직이는 것처럼 보입니다. 셋째, 켄타우루스자리의 프록시마 성은 우리 태양계에서 가장 가까운 별입니다. 넷째, 북극성은 소 북두칠성이라 불리는 별 패턴의 일부입니다. 그렇다면 태양이 매일 하늘을 가로질러 움직이는 것처럼 보이는 이유는 무엇일까요? 지구가 자전축을 중심으로 회전하기 때문입니다.
score: 0.36437815 source: 당신은 금성에 살고 있고, 당신의 망원경이 금성의 두꺼운 구름을 볼 수 있다고 가정한다면, 지구의 달을 관찰할 수 있습니다. 지구의 달은 월식과 만월을 거치며 다양한 단계를 보여줍니다. 여기서는 월식이 아닌 단계를 중심으로 설명하겠습니다.

첫 번째로, 상현 단계를 볼 수 있습니다. 이 단계에서는 달이 점점 커지면서 반달 모양을 보여줍니다. 달의 오른쪽 절반은 밝고, 왼쪽 절반은 어둡습니다. 이는 달이 점점 지구와 태양 사이의 각도가 커지면서 발생하는 현상입니다.

두 번째로, 망 단계를 볼 수 있습니다. 이 단계에서는 달이 지구와 태양 사이에 정확히 위치하여 완전한 원형을 보여줍

In [ ]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.64321625 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 0.63613594 source: 토성은 왜 질량이 더 작음에도 불구하고 거의 목성만큼 큰가요?

토성과 목성은 둘 다 태양계에서 가장 큰 행성 중 하나입니다. 하지만 토성은 목성보다 질량이 작습니다. 그렇다면 왜 토성은 목성만큼 큰 것일까요?

이 질문에 대한 답은 목성의 더 큰 질량이 목성을 더 압축하여 밀도를 증가시킨다는 것입니다. 목성은 토성보다 더 많은 물질을 포함하고 있으며, 이로 인해 목성은 더 큰 중력을 가지고 있습니다. 이 중력은 목성의 물질을 압축시키고, 결과적으로 목성의 밀도를 증가시킵니다.

반면에 토성은 목성보다 더 적은 물질을 포함하고 있습니다. 따라서 토성은 목성보다 더 낮은 중력을 가지고 있으며, 이로 인해 토성은 더 큰 크기를 유지할 수 있습니다. 토성은 목성보다 더 낮은 밀도를 가지고 있지만, 그 크기는 목성과 거의 비슷합니다.

이렇게 토성은 목성보다 질량이 작지만, 목성의 더 큰 질량이 목성을 더 압축하여 밀도를 증가시킨다는 이유로 거의 목성만큼 큰 것입니다.
score: 0.60802525 source: 달을 보는 것이 태양을 보는 것보다 왜 더 안전한가요?

달을 보는 것이 태양을 보는 것보다 더 안전한 이유는 달이 덜 밝기 때문입니다. 태양은 매우 강력한 광원이며, 직접 눈에 비추면 시력을 손상시킬 수 있습니다. 그러나 달은 태양으로부터 받는 빛을 반사하기 때문에 태양보다 훨씬 덜 밝습니다. 따라서 달을 보

In [ ]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = mix_retrieve(test_query,5)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 1.3067844 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 0.8667359 source: 태양은 지구에 가장 가까운 별이기 때문에 다른 별들보다 더 크게 보입니다. 태양은 우리 은하계에서 가장 큰 별은 아니지만, 지구로부터 가까운 거리 때문에 크게 보이는 것입니다. 태양은 약 1,390,000km의 지름을 가지고 있으며, 이는 지구의 약 109배에 해당합니다. 이러한 크기 때문에 태양은 우리에게 매우 밝게 보이는 것입니다. 태양은 또한 지구와의 거리에 따라 크기가 변하지 않기 때문에 항상 크게 보입니다. 그러나 실제로는 태양보다 훨씬 더 큰 별들이 존재합니다. 이러한 별들은 지구로부터 매우 멀리 떨어져 있기 때문에 작게 보이는 것입니다. 따라서, 태양이 가장 큰 별처럼 보이는 것은 태양이 지구에 가까워서인 것입니다.
score: 0.83698595 source: 태양의 코로나를 볼 수 없는 이유는 너무 흩어져 있기 때문입니다. 태양의 코로나는 태양의 외부 대기로서, 태양의 표면에서 멀리 떨어져 있습니다. 이 코로나는 태양의 높은 온도로 인해 밝게 빛나지만, 그 밝은 빛은 태양의 표면에서 나오는 빛에 가려져 보이지 않습니다. 따라서, 우리는 개기 일식 동안에만 태양의 코로나를 볼 수 있습니다. 개기 일식은 태양과 달이 정확히 일직선에 위치할 때 발생하며, 태양의 표면이 가려져 코로나만이 노출되기 때문에 우리는 태양의 코로나를 관찰할 수 있습니다. 그러나 개기 일식은 특정 지역에서만 볼 수 있으며, 그 기간도

/tmp/ipykernel_1026150/2586751028.py:111: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body=body)


## RAG 구현

준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [ ]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "sk-"
# os.environ["OPENAI_API_KEY"] = "sk-"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"
# llm_model = "gpt-4-1106-preview"


In [ ]:
# RAG 구현에 필요한 Question Answering을 위한 LLM 프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 지식에 관해 질문하는 경우에는 반드시 search 함수를 호출한다.
- 나머지 메시지에는 함수 호출 없이 적절한 대답을 생성한다.
"""
# 원본 
# 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
# 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.

# test 1
# - 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 반드시 search function을 호출한다.
# - 과학 상식과 관련되지 않은 나머지 대화 메시지에는 반드시 search function 없이 적절한 대답을 생성한다.

# test 2 : 0.4379
# - 대화 메시지나 질문이 들어오면 먼저 해당 메시지가 과학 상식에 관한 질문인지 판단합니다.
# - 만약 질문이 과학 상식에 관한 것이라면, 미리 검색 엔진에 색인된 문서들 중에서 적합한 문서들을 추출합니다.
# - 추출된 문서를 기반으로 적절한 답변을 생성합니다.
# - 질문이 과학 상식과 관련이 없는 경우에는 검색 엔진을 사용하지 않고, 적절한 답변을 바로 생성합니다.

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "question": {
                                    "type": "string",
                                    "description": "User's question in Korean. Full message if the message is single-turn."
                                }
                },
                "required": ["question"],
                "type": "object"
            }
        }
    },
]

In [ ]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"question": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            # timeout=10
            timeout=60
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("question")

        # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
        search_result = mix_retrieve(standalone_query, 3)
        # search_result = dense_retrieve(standalone_query, 3)

        response["question"] = standalone_query
        retrieved_context = []
        for i,rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    # timeout=30
                    timeout=60
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response



In [ ]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 5:
            #   break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], 
                      "question": response["question"], 
                      "topk": response["topk"], 
                      "answer": response["answer"], 
                    #   "references": response["references"]
                      }
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [ ]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../data/eval.jsonl", "mix_retrival_sentence-distiluse-base-multilingual-cased-v2.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


/tmp/ipykernel_1026150/2586751028.py:111: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body=body)


Answer: 나무의 분류를 조사하기 위한 방법은 다양합니다. 먼저 나무의 잎 모양, 줄기 모양, 꽃과 열매의 유무 등을 관찰하여 나무를 분류할 수 있습니다. 또한 나무의 생태학적 특성을 고려하여 분류할 수도 있습니다. 또한 나무의 생육지역, 생태적 특성, 환경 등을 고려하여 분류할 수 있습니다. 이러한 분류는 나무의 생태학적 특성과 연구에 필요한 정보를 제공합니다. 또한 나무의 분류는 생태학적 특성과 연구에 필요한 정보를 제공합니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 죄송합니다, 현재 제가 찾을 수 있는 최신의 각 나라의 공교육 지출 현황에 대한 정보가 부족합니다. 더 나은 답변을 위해서는 보다 신뢰할 만한 출처의 정보가 필요합니다. 다른 질문이 있으시면 도와드릴 수 있는 다른 주제가 있을지도 모릅니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 기억 상실증은 다양한 원인에 의해 발생할 수 있습니다. 일반적으로 기억 상실증은 뇌의 기능이나 구조적인 변화로 인해 발생할 수 있습니다. 이러한 변화는 외상, 질병, 노화, 유전적 요인, 스트레스, 알코올 또는 약물 남용 등 다양한 요인에 의해 발생할 수 있습니다. 따라서 기억 상실증의 원인은 다양하며, 각각의 경우에 따라 다를 수 있습니다.

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 통학 버스는 주로 학생들이 학교로 통학할 때 이용하는 버스로, 학교 시간에 맞춰 운행되며 학생들의 안전과 편의를 고려한 운행이 이루어집니다. 또한 학교 

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'failed to create query: the query vector has a different dimension [512] than the index vectors [384]')

In [ ]:
!wc -l sample_submission.csv

220 sample_submission.csv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


#Reference

## Required Package

openai==1.7.2 <br>
elasticsearch==8.8.0 <br>
sentence_transformers==2.2.2 <br>



## 콘텐츠 라이선스

저작권 : <font color='blue'> <b> ©2023 by Upstage X fastcampus Co., Ltd. All rights reserved.</font></b>

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 업스테이지 및 패스트캠퍼스에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. </b>